In [1]:
%matplotlib widget

<html>
    <h1>Libraries used :</h1>
    <ul style="list-style-type: none;">
        <li>
             <img width=150px height=150px  src="https://upload.wikimedia.org/wikipedia/commons/thumb/5/54/Sympy_logo.svg/1200px-Sympy_logo.svg.png"  alt="Sympy">
        </li>
        <br>
        <li>
             <img width=300px height=300px  src="https://matplotlib.org/_static/logo2.png" alt="Matplotlib">
        </li>
        <br>
        <li>
             <img width=200px height=200px src="https://upload.wikimedia.org/wikipedia/commons/thumb/1/1a/NumPy_logo.svg/1200px-NumPy_logo.svg.png" alt="Numpy">
        </li>
        <br>
        <li>        
            <img width=200px height=200px  src="https://www.fullstackpython.com/img/logos/scipy.png" class="rounded" alt="Scipy">
        </li>
        <br>
        <li>
             <img width=400px height=400px  src="https://pandas.pydata.org/_static/pandas_logo.png" class="rounded" alt="Pandas">
        </li>
      </ul>

In [2]:

import numpy as np
import sympy as sm
from scipy.sparse import diags
from scipy.integrate import RK45, solve_ivp
import scipy.sparse as sp
import scipy.sparse.linalg as la
import matplotlib.pyplot as plt
import matplotlib
sm.init_printing()

import time
import os
from functools import lru_cache
import math
import pandas as pd
from tqdm import tqdm_notebook


\begin{equation}
\underline {System \ of \ equations :}
\\\ u_t + u  u_x + v_x + \beta u_{xx} = 0 
\\ v_t + (uv)_x - \beta v_{xx} = 0 
\end{equation}

\begin{equation}
\underline {  Galerkin \ Solution :}
\\ u^h = 
\sum_{i= 0}^{n} a_i(t)  \phi_i(x) 
\end{equation}

\begin{equation}
\underline {System \ of \ equations :}
\\\ \int_0^1 { (w u_t + wu  u_x + wv_x + \beta w u_{xx})} \ dx= 0 
\\ \int_0^1 { (wv_t + w (uv)_x - \beta w v_{xx})} \ dx= 0 
\end{equation}



<h3 align="center" style="color:#FFE647;">Equation  (i) </h3> 

\begin{equation}
\\\ \int_0^1  (w u_t  + wuu_x + wv_x ) \ dx  - \beta \int_0^1 {w_x u_x} \ dx + \beta \ (w(1) u_x(1) - w(0) u_x(0)) = 0 
\\  (A \gamma^{'} + \gamma^TB\gamma +C\delta)\hspace{5mm} - \hspace{15mm}\beta  D \gamma \hspace{10mm} + \hspace{10mm} \beta E \gamma \hspace{10mm}   = 0 
\\ w = {\phi_j}
\end{equation}

<h3 align="center" style="color:#FF3647;">Equation  (ii) </h3> 

\begin{equation}
\\\ \int_0^1  (w v_t  + w(uv)_x ) \ dx + \beta \int_0^1 {w_x v_x} \ dx - \beta \ (w(1) v_x(1) - w(0) v_x(0)) = 0 
\\  (A \delta^{'} + \gamma^TG\delta)\hspace{5mm}  + \hspace{15mm}\beta  D \delta \hspace{10mm} - \hspace{10mm} \beta E \delta =0 
\\ G = 2B
\\ w = {\phi_j}
\end{equation}

<html>
    <h1>

In [3]:
from IPython.core.display import HTML
from IPython.display import IFrame
IFrame(src='https://www.hindawi.com/journals/ijde/2008/243459/', width=990, height=600)

In [4]:


## Gamma -----> u(x, t)

## Delta -----> v(x, t)

class Matrices:
    
    #A *gamma - beta(D *) +beta(E(1 - 0)gamma) = 0
    
    def __init__(self, N=10):
        
        self.N = N  #no of basis functions
        self.shape = (N+1, N+1) #shape of matrices
        self.h = 1/N #distance step
        self.domain = np.linspace(0, 1, N+1) #domain 0-1
        self.beta = -1  #constant 
        self.time_step = self.h**2 #time step 
        self.ld = .005
        self.alpha = .5
        self.beta = 1
        self.x0 = 10
        self.k = .2
        
        self.A = (self.h) * sp.csc_matrix(diags([       [1/6 for i in range(N)],      
                                        [1/3] + [2/3]*(N-1) + [1/3], 
                                             [1/6 for i in range(N)] ], 
                                                   [1, 0, -1]), dtype=np.float32)  #phi phi
        #+
        self.B = np.array([self.matB(i) for i in range(N+1)]) #non linear
        #+
        
        self.C = sp.csc_matrix(diags([       [1/2 for i in range(N)],   
                                        [-1/2] + [0]*(N-1) + [1/2], 
                                             [-1/2 for i in range(N)] ], 
                                                   [1, 0, -1]), dtype=np.float32)  #phi phi'
        

        #-
        self.D = (1/self.h) * sp.csr_matrix(diags([      [-1 for i in range(N)],    
                                      [1] + [2]*(N-1) + [1], 
                                        [-1 for i in range(N)]],  
                             [1, 0, -1] ), dtype=np.float32)  # phi' phi'
                #+                
        row = np.array([0, 0, N, N])
        col = np.array([0, 1, N-1, N])
        data = np.array([1, -1, -1, 1 ])
        self.E = (1/self.h) *sp.csr_matrix((data, (row, col)), shape=self.shape, dtype=np.float32)    # non integrated
    
    
        #Set initial conditions here
        self.gamma = self.ld - 2 * self.k *(self.alpha + self.beta**2) **.5 * (np.tanh(self.k * (self.domain + self.x0)))**-1
        self.delta = - 2* self.k**2 *(self.alpha + self.beta**2 + self.beta * (self.alpha + self.beta**2) **.5) * (np.sinh(self.k * (self.domain + self.x0)))**-2
        
        #Set solution here
        self.exact_gamma = lambda t : self.ld -2 * self.k *(self.alpha + self.beta**2) **.5 * (np.tanh(self.k * (self.domain + self.x0) - self.ld * t))**-1 
        self.exact_delta = lambda t : -2 * self.k**2 *(self.alpha + self.beta**2 + self.beta * (self.alpha + self.beta**2) **.5) * (np.sinh(self.k * (self.domain + self.x0) - self.ld * t))**-2
        
    def matB(self,i):
            if i == 0 :
                row = np.array([0, 0, 1, 1])
                col = np.array([0, 1, 0, 1])
                data = np.array([-1/3, 1/3, -1/6, 1/6])
                return sp.csr_matrix((data, (row, col)), shape=self.shape,dtype=np.float32)        
            elif i == self.N:
                row = np.array([self.N-1, self.N - 1, self.N, self.N])
                col = np.array([self.N-1, self.N , self.N -1, self.N])
                data = np.array([-1/6, 1/6, -1/3, 1/3])
                return sp.csr_matrix((data, (row, col)), shape=self.shape, dtype=np.float32)   
            else:
                row = np.array([i-1]*3 + [i]*3 + [i+1]*3)
                col = np.array([col for col in range(i-1, i+1 + 1)]*3, dtype=np.float32)
                data = np.array([-1/6, 1/6, 0, -1/3, 0, 1/3, 0, -1/6, 1/6 ])
                return sp.csr_matrix((data, (row, col)), shape=self.shape, dtype=np.float32)    
            
                      

                

In [5]:
class Methods(Matrices): 
    
    def __init__(self, num_basis = 10, method="Adam"):
        
        super().__init__(num_basis)
        eval("self."+method+"()")
        
        self.algo.y[0, 0], self.algo.y[0, 1] = self.exact_gamma(self.algo.t[0])[0],\
                   self.exact_gamma(self.algo.t[1])[0]
        self.algo.y[self.N, 0], self.algo.y[self.N, 1] = self.exact_gamma(self.algo.t[0])[-1], \
                   self.exact_gamma(self.algo.t[1])[-1]
        self.algo.y[self.N+1, 0], self.algo.y[self.N+1, 1] = self.exact_delta(self.algo.t[0])[0], \
                    self.exact_delta(self.algo.t[1])[0]
        self.algo.y[-1, 0], self.algo.y[-1, 1] =  self.exact_delta(self.algo.t[0])[-1], \
                   self.exact_delta(self.algo.t[1])[-1]
        
        self.approx_half_gamma, self.approx_half_delta, self.approx_one_gamma, self.approx_one_delta = \
                             np.split(self.algo.y.flatten("F"), 4)  

        self.exact_half_gamma, self.exact_half_delta, self.exact_one_gamma, self.exact_one_delta = \
            self.exact_gamma(.5),  self.exact_delta(.5), self.exact_gamma(1), self.exact_delta(1)
               
               

        self.error_half_gamma = math.sqrt(sum((self.approx_half_gamma - self.exact_half_gamma)**2))
        self.error_one_gamma = math.sqrt(sum((self.approx_one_gamma - self.exact_one_gamma)**2))

        self.error_half_delta = math.sqrt(sum((self.approx_half_delta - self.exact_half_delta)**2))
        self.error_one_delta = math.sqrt(sum((self.approx_one_delta - self.exact_one_delta)**2))
        
        
        
    def fun(self, t, y):

        gamma, delta = np.split(y, 2)
        gamma[0] = self.exact_gamma(t)[0]  # x boundary conditions
        gamma[-1] = self.exact_gamma(t)[-1]
        delta[0] = self.exact_delta(t)[0] 
        delta[-1] = self.exact_delta(t)[-1]
        gamma = gamma.reshape(-1, 1)
        delta = delta.reshape(-1, 1)
        return np.concatenate([(la.inv(self.A).dot(-self.C.dot(delta)-((gamma.T).dot(self.B.flatten())[0]+self.beta*(self.D - self.E)).dot(gamma) )).flatten(), 
                               (la.inv(self.A).dot((-(2*gamma.T).dot(self.B.flatten())[0] + self.beta*(-self.D + self.E)).dot(delta))).flatten()]) 
                 
    
    def Adam(self):

        self.algo = solve_ivp(self.fun,t_span=(0, 1),  y0=np.concatenate([self.gamma, self.delta]), t_eval= [0.5, 1], vectorized=True, first_step = self.h, method="LSODA")
    

In [6]:
points = [5*j for j in range(2, 30)]

half = pd.DataFrame(data =None, columns=["Approximate u", "Exact u", "$ ||.||_2 \ Error u $", "Approximate v", "Exact v", "$ ||.||_2 \ Error v $"])
one = pd.DataFrame(data =None, columns=["Approximate u", "Exact u", "$ ||.||_2 \ Error u $", "Approximate v", "Exact v", "$ ||.||_2 \ Error v $"])

for i in tqdm_notebook(points):
    heateqn = Methods(i, method="Adam")
    

    half.loc[i] = [heateqn.approx_half_gamma, heateqn.exact_half_gamma, heateqn.error_half_gamma, heateqn.approx_half_delta, heateqn.exact_half_delta, heateqn.error_half_delta]

    one.loc[i] = [heateqn.approx_one_gamma, heateqn.exact_one_gamma, heateqn.error_one_gamma, heateqn.approx_one_delta, heateqn.exact_one_delta, heateqn.error_one_delta]

KeyboardInterrupt: 

In [ ]:
x = np.log(one.index.tolist())
y_u = np.log(one[ "$ ||.||_2 \ Error u $"].tolist())
y_v = np.log(one[ "$ ||.||_2 \ Error v $"].tolist())

In [ ]:
x

In [ ]:
y_u

In [ ]:
y_v

In [ ]:
plt.plot(x, y_u)

In [7]:
errors

NameError: name 'errors' is not defined

In [8]:
half

,Approximate u,Exact u,$ ||.||_2 \ Error u $,Approximate v,Exact v,$ ||.||_2 \ Error v $
10,"[-0.5032717029250897, -0.5028121832941083, -0....","[-0.5032717029250897, -0.5025382867475189, -0....",0.001365,"[-0.016657386264782463, -0.01582843436273156, ...","[-0.016657386264782463, -0.01598073086191085, ...",0.000908
15,"[-0.5032717029250897, -0.5029734220450923, -0....","[-0.5032717029250897, -0.5027793875451584, -0....",0.001680,"[-0.016657386264782463, -0.016089648872508494,...","[-0.016657386264782463, -0.016203064504127722,...",0.001139
20,"[-0.5032717029250897, -0.5030197155744403, -0....","[-0.5032717029250897, -0.5029011931206973, -0....",0.001959,"[-0.016657386264782463, -0.01621927613068761, ...","[-0.016657386264782463, -0.01631542894790494, ...",0.001327
